## モジュールのインストール

```zsh
% pip install selenium
```

## 基本

### Chromeドライバーの起動

```python
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Chromeドライバーのオプション
options = Options()
options.add_argument('--disable-gpu');
options.add_argument('--disable-extensions');
options.add_argument('--proxy-server="direct://"');
options.add_argument('--proxy-bypass-list=*');
options.add_argument('--start-maximized');
options.add_argument('--headless'); # ヘッドレスモードON

# Chromeドライバーを起動
driver = webdriver.Chrome(executable_path='Chromeドライバーのパス', options=options)
```

### Webページにアクセス

```python
# Webページにアクセスする
driver.get('URL')
```

### ページ全体の操作

逆引き | コマンド | 戻り値
:-- | :-- | :--
ページのタイトルを取得する | `driver.title` | `str`
ページのURLを取得する | `driver.current_url` | `str`
ソースコードを取得する | `driver.page_source` | `str`
スクリーンショットを撮る | `driver.save_screenshot('保存先のパス')` | 
ブラウザバック | `driver.back()` | 
ブラウザバックの取り消し | `driver.forward()` | 
ページをリロードする | `driver.refresh()` | 
ウィンドウハンドルの一覧を取得する | `driver.window_handles` | `list`
操作するウィンドウを切り替える | `driver.switch_to.window('ウィンドウハンドル')` | 
Cookieをセットする | `driver.add_cookie({'name': 'Cookieの名前', 'value': 'Cookieの値'})` | 
ページを閉じる | `driver.close()` | 
ブラウザを終了する | `driver.quit()` | 

### 要素の指定

```python
# id属性
element = driver.find_element_by_id('id')

# name属性
element = driver.find_element_by_name('name')

# CSSセレクタ
element = driver.find_element_by_css_selector('CSSセレクタ')
```

### 要素の操作

逆引き | コマンド | 戻り値
:-- | :-- | :--
テキストを取得する | `element.text` | `str`
属性の値を取得する | `element.get_attribute('属性名')` | `str`
テキストを入力する | `element.send_keys('テキスト')` | 
テキストを削除する① | `element.clear()` | 
テキストを削除する② | `[element.send_keys(Keys.BACK_SPACE) for _ in range(len(element.text))]` | 
要素をクリックする① | `element.click()` | 
要素をクリックする② | `driver.execute_script('arguments[0].click();', element)` | 
特殊キーを押す(ENTERキー) | `element.send_keys(Keys.ENTER)` | 
特殊キーを押す(BACKSPACEキー) | `element.send_keys(Keys.BACK_SPACE)` | 
特殊キーを押す(ESCAPEキー) | `element.send_keys(Keys.ESCAPE)` | 
特殊キーを押す(TABキー) | `element.send_keys(Keys.TAB)` | 
要素のx座標を取得 | `element.location['x']` | `int`
要素のy座標を取得 | `element.location['y']` | `int`

### その他の操作

#### 待機処理

##### 暗黙的な待機

全ての`find_element`等の処理時に、要素が見つかるまで指定した最大時間待機させるようにすることができます。

```python
# 暗黙的な待機
driver.implicitly_wait(10) # 秒
```

##### 明示的な待機

任意のHTML要素が**特定の状態**になるまで待つ明示的な待機時間を設定することができます。

コマンド | 状態
:-- | :--
`alert_is_present` | Alertが表示されるまで待機
`element_to_be_clickable` | 要素がクリック出来る状態になるまで待機
`visibility_of_element_located` | 指定した要素が表示されるまで待機
`invisibility_of_element_located` | 指定した要素が非表示になるまで待機
`text_to_be_present_in_element` | 指定したテキストが表示されるまで待機
`presence_of_element_located` | 指定した要素がDOM上に現れるまで待機

```python
# 指定した要素が表示されるまで、30秒待機する
element = WebDriverWait(driver, 30).until(
	EC.visibility_of_element_located((By.CSS_SELECTOR, 'CSSセレクタ'))
)
```

##### 最終手段

```python
import time

time.sleep(30) # 秒
```

#### JavaScriptのDOM操作

逆引き | コマンド | 戻り値
:-- | :-- | :--
ページの幅を取得 | `driver.execute_script('return document.body.scrollWidth')` | `int`
ページの高さを取得 | `driver.execute_script('return document.body.scrollHeight')` | `int`
ページトップから垂直方向にスクロールする | `driver.execute_script('window.scrollTo(0, ピクセル);')` | 
ページトップから一番下までスクロールする（Ajax非対応）| `driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')` | 
相対位置に垂直方向にスクロールする | `driver.execute_script('window.scrollBy(0, ピクセル);')` | 
ページトップからの垂直方向のスクロール量を取得する | `driver.execute_script('return window.pageYOffset')` | `int`
要素をクリックする | `driver.execute_script('arguments[0].click();', element)` | 
`display: none;`で隠された要素を表示する | `driver.execute_script('arguments[0].style.display="";', element)` | 

## 応用

### BeautifulSoup × Selenium

```python
from bs4 import BeautifulSoup

# 
# 中略 
#

# ソースコードを取得
html = driver.page_source

# HTMLをパースする
soup = BeautifulSoup(html, 'lxml')
```

### Ajaxページを一番下までゆっくりスクロールする

```python

# ページの高さを取得
height = driver.execute_script('return document.body.scrollHeight')

# ループ処理で少しづつスクロール移動し、Ajaxページの一番下までゆっくりスクロールする
i = 1
while i < height:
    driver.execute_script(f'window.scrollTo(0, {i});')
    height = driver.execute_script('return document.body.scrollHeight')
    i += 1
```

### ファイルの保存場所を変更する

```python
# ファイルのデフォルトの保存先を変更する
driver.command_executor._commands['send_command'] = (
    'POST',
    '/session/$sessionId/chromium/send_command'
)
params = {
    'cmd': 'Page.setDownloadBehavior',
    'params': {
        'behavior': 'allow',
        'downloadPath': '保存先のディレクトリ'
    }
}
driver.execute('send_command', params=params)
```

### BASIC認証を突破する

```python
# BASIC認証のID、PWとURLをセットする
ID = 'id'
PW = 'password'
URL = 'url'

# BASIC認証を突破する
driver.get(f'{ID}:{PW}@{URL}')
```

### 2段階認証を突破する

[Python+Seleniumで2段階認証（6桁のパスコード）を突破する全手順](https://tanuhack.com/selenium-2step-authentication/)

### reCAPTCHAを突破する

[Python+Seleniumで『reCAPTCHA』を突破する方法](https://tanuhack.com/pr-2captcha/)